In [1]:
x = 3

In [2]:
print(x)

3


In [3]:
y = 4

In [4]:
x * y

12

In [5]:
x > y

False

In [6]:
x < y

True

In [7]:
x >= y

False

In [8]:
x <= y

True

In [9]:
x = y

In [10]:
print(x)

4


In [11]:
x == y

True

In [15]:
list_values = ['Meredith', 'Eric', 'Matt', 'Molly', 'Joe']

for name in list_values:
    print(name)

Meredith
Eric
Matt
Molly
Joe


In [17]:
for name in list_values:
    if name == "Molly":
        print(name)
    elif name == "Eric":
        print(name)
    else:
        print("The name's not molly!")

The name's not molly!
Eric
The name's not molly!
Molly
The name's not molly!


In [19]:
for name in list_values:
    if name.startswith('M'):
        print(name)

Meredith
Matt
Molly


In [21]:
M_name_list = []

for name in list_values:
    if name.startswith('M'):
        M_name_list.append(name)
        
print(M_name_list)

['Meredith', 'Matt', 'Molly']


In [22]:
M_name_list = [name for name in list_values if name.startswith('M')]

print(M_name_list)

['Meredith', 'Matt', 'Molly']


In [28]:
def get_mnames_from_list(in_name_list):
    M_name_list = []

    for name in in_name_list:
        if name.startswith('M'):
            M_name_list.append(name)
    return M_name_list

In [29]:
name_list_2 = ['Mia', 'Mason', 'Liam', 'Noah']
name_list_3 = ['Mark', 'Bill', 'George']
name_list_4 = ['Marcos', 'Frank', 'Dennis']

In [33]:
get_mnames_from_list(name_list_4)

['Marcos']

In [36]:
new_mname_list = get_mnames_from_list(name_list_4)

In [37]:
new_mname_list

['Marcos']

In [1]:
#Refactoring

In [ ]:
import os
import arcpy
import zipfile

arcpy.env.overwriteOutput = True

def create_new_folders_from_list(in_folder_list):
    """Create new folders with paths from a list"""
    for folder in in_folder_list:
        if not os.path.exists(folder):
            os.mkdir(folder)

example_folder = r".\Example_Data"
extracted_folder = r".\Example_Data\Extracted_SHP"
output_folder = r".\Example_Data\Output_Folder"

create_new_folders_from_list([example_folder, extracted_folder, output_folder])

zipfile_list = [f for f in os.listdir('.') if f.endswith('.zip')]
for zf in zipfile_list:
    zfile = zipfile.ZipFile(zf)
    zfile.extractall(extracted_folder)

arcpy.env.workspace = extracted_folder
shp_list = arcpy.ListFeatureClasses(feature_type = "Point")

for pt_shp in shp_list:
    print('Updating', pt_shp)
    output_shp = os.path.join(output_folder, pt_shp[:-4] + '_spjoin.shp')
    arcpy.analysis.SpatialJoin(pt_shp, 'SF_Neighborhoods.shp', output_shp, match_option="COMPLETELY WITHIN")

In [ ]:
#Problem 1

arcpy.env.workspace = r'.\Example_Data\Extracted_SHP'
shp_list = arcpy.ListFeatureClasses()

for shp in shp_list:
    print(f'{shp} has {arcpy.GetCount_management(shp)} records!')

In [ ]:
#Problem 2

sf_business_shp = r'.\Example_Data\Extracted_SHP\SF_Businesses.shp'

with arcpy.da.SearchCursor(sf_business_shp, "dba_name") as cursor:
    for row in cursor:
        if "Pizza" in row[0]:
            print(row[0])

In [ ]:
#Problem 3

sf_business_shp = r'.\Example_Data\Extracted_SHP\SF_Businesses.shp'

arcpy.AddField_management(sf_business_shp, "Food_Type", "TEXT")

fields = ["dba_name", "Food_Type"]
categories = ["Pizza", "Ice Cream", "Coffee", "Thai", "Mexican"]

with arcpy.da.UpdateCursor(sf_business_shp, fields) as cursor:
    for row in cursor:
        for cat in categories:
            if cat in row[0]:
                row[1] = cat
        cursor.updateRow(row)

In [ ]:
#Problem 4

arcpy.env.workspace = r'.\Example_Data\Extracted_SHP'

output_folder_practprob2 = r".\Example_Data\output_folder_practprob2"
if not os.path.exists(output_folder_practprob2):
    os.mkdir(output_folder_practprob2)

locations_sp = r"memory\locations_sp"
arcpy.analysis.SpatialJoin('Locations_of_Interest.shp', 'SF_Neighborhoods.shp', locations_sp, match_option="COMPLETELY WITHIN")

locations_sp_buffer = r"memory\locations_sp_buffer"
arcpy.analysis.Buffer(locations_sp, locations_sp_buffer, "1 Mile")

shp_list = arcpy.ListFeatureClasses()
for shp in shp_list:
    in_features = [locations_sp_buffer, shp]
    out_features = os.path.join(output_folder_practprob2, f'{shp[:-4]}_intersect.shp')
    arcpy.analysis.Intersect(in_features, out_features)